In [1]:
#based on
#https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/
#https://datamarket.com/data/set/22r0/sales-of-shampoo-over-a-three-year-period#!ds=22r0&display=line

In [2]:
DATA_DIR = "/home/ubuntu/fastai/data/"

In [3]:
# load and plot dataset
from pandas import read_csv
from pandas import datetime
from pandas import DataFrame
from pandas import concat
from matplotlib import pyplot
from pandas import Series

from math import sqrt

from sklearn.metrics import mean_squared_error

In [4]:
# load dataset
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')


In [5]:
series = read_csv(DATA_DIR+'shampoo_sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
# summarize first few rows
print(series.head())
print (series.shape)

Month
1901-01-01    266.0
1901-02-01    145.9
1901-03-01    183.1
1901-04-01    119.3
1901-05-01    180.3
Name: Sales, dtype: float64
(36,)


In [6]:
# create a differenced series
def difference(dataset, interval=1):
    #calculates difference between each element in dataset and element @ interval before.
    #returns panda series with # interval less rows.
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)

In [7]:
# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

In [8]:
# transform to be stationary
differenced = difference(series, 1)
print(differenced.head())
print (series.shape)
print (differenced.shape) 
print (type(differenced))


0   -120.1
1     37.2
2    -63.8
3     61.0
4    -11.8
dtype: float64
(36,)
(35,)
<class 'pandas.core.series.Series'>


In [15]:
print (type(series), series.shape, series[0:5])
type(differenced), differenced.shape

<class 'pandas.core.series.Series'> (36,) Month
1901-01-01    266.0
1901-02-01    145.9
1901-03-01    183.1
1901-04-01    119.3
1901-05-01    180.3
Name: Sales, dtype: float64


(pandas.core.series.Series, (35,))

In [10]:
# invert transform
inverted = list()
for i in range(len(differenced)):
	value = inverse_difference(series, differenced[i], len(series)-i)
	inverted.append(value)
inverted = Series(inverted)
print(inverted.head())

0    145.9
1    183.1
2    119.3
3    180.3
4    168.5
dtype: float64
